In [449]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import sys
import re
import matplotlib.pyplot as plt
import seaborn as sns
import geopy.distance
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 10000)
pd.options.display.max_rows = 10000
# from google.colab import drive
# drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


**Loading the dataset for December**

In [450]:
# data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/PS2/data/december/listings.csv")
data = pd.read_csv("data/december/listings.csv")

**Droping unnecessary columns. Some of them has a lot NaNs. Some of them are just meaningless for prediction. Or is similar to other feature**

In [451]:
drops = ["calendar_updated", "neighbourhood", "host_response_time", "host_response_rate",
         "host_neighbourhood", "has_availability","host_verifications", "host_has_profile_pic", "host_total_listings_count",
    "license",
    "neighbourhood_group_cleansed",
    "bathrooms",
    "host_thumbnail_url",
    "host_picture_url",
    "listing_url",
    "picture_url",
    "host_url",
    "last_scraped",
    "description", "calendar_last_scraped",
    "neighborhood_overview",
    "host_about",
    "name", "host_location", "host_id", "host_name"]
data.drop(columns=drops, inplace=True)

**Converting percentage values to float**

In [452]:
data['host_acceptance_rate'] = data['host_acceptance_rate'].str.rstrip("%").astype(float)/100

**Cleaning the target (price)**

In [453]:
data['price'] = data['price'].str.replace(r'[$,]', '').astype(float)

**Checking for the outliers. Just 2. Seems OK**

In [454]:
# prices == 0
# prices >25k
data[data["price"]>25000]["price"]

798     26696.0
1144    71536.0
Name: price, dtype: float64

**Converting to binary variables**

In [455]:
data['host_is_superhost'] = data['host_is_superhost'].apply(lambda x: 1 if x == 't' else (0 if x == 'f' else x))

In [456]:
data['host_identity_verified'] = data['host_identity_verified'].apply(lambda x: 1 if x == 't' else (0 if x == 'f' else x))

In [457]:
data["host_identity_verified"].fillna(0, inplace=True)

In [458]:
data['instant_bookable'] = data['instant_bookable'].apply(lambda x: 1 if x == 't' else (0 if x == 'f' else x))

**Cleaning the string in amenities feature**

In [459]:
data['amenities'] = data['amenities'].apply(lambda x: re.sub(r'\\u\w{4}', '', re.sub(r'\\', '', re.sub(r'\[', '',  re.sub(r'\]', '', re.sub(r'"', '', re.sub(r',', '', x)))))  ))

**Collecting all words in a list for each row**

In [460]:
words = []
data['amenities'].apply(lambda x:  words.append(x.split(" ")))
flat_list = []
for sub_list in words:
    for item in sub_list:
        flat_list.append(item)

**Droping meaningless and less frequent words. Then choosing the most frequent ones**

In [461]:
ab = pd.DataFrame(pd.Series(flat_list).value_counts())
ab.columns = ["counts"]
ab = ab[ab["counts"]>600]
ba = ab.transpose()
ba = ba.drop(columns=["and", "allowed", "Private", "Essentials","silverware",\
                      "Long", "Hair", "stays", "term", "Bed", "Shampoo", "Cooking", "basics", "u2013",\
                      "Paid", "Carbon", "monoxide", "maker"
                     ], axis=1)
findd = ba.columns[:18]

**Creating dummies**

In [462]:
for ind, row in data.iterrows():
    for i in list(set(row["amenities"].split(" ")) & set(findd)):
        data.at[ind, i] = 1

In [463]:
for i in findd:
    data[i].fillna(0, inplace=True)

**Creating dummies for property type**

In [464]:
data["property_type"].value_counts().head(15)

Entire rental unit                   2384
Entire condo                         1284
Private room in rental unit           520
Entire home                           432
Private room in bed and breakfast     294
Entire townhouse                      213
Entire loft                           186
Private room in condo                 135
Houseboat                             132
Private room in home                  130
Room in boutique hotel                122
Room in hotel                         117
Private room in houseboat             102
Private room in guest suite            99
Private room in townhouse              91
Name: property_type, dtype: int64

In [465]:
for_ = ['Private room in houseboat', 'Boat', 'Entire rental unit',
        'Private room in bed and breakfast', 'Entire villa','Private room in townhouse',
        'Private room in home', 'Entire condo', 'Entire home', 'Houseboat', 'Private room in guest suite',
        'Shared room in hostel', 'Entire loft']
specific_dummies = pd.get_dummies(data['property_type'][data['property_type'].isin(for_)], prefix="proptype")
data = pd.concat([data, specific_dummies], axis=1)

In [466]:
for i in for_:
    data["proptype_"+i].fillna(0, inplace=True)

**Creating dummies for room type**

In [467]:
specific_dummies = pd.get_dummies(data['room_type'], prefix="room_type", drop_first=True)
data = pd.concat([data, specific_dummies], axis=1)

In [468]:
specific_dummies = pd.get_dummies(data['neighbourhood_cleansed'], prefix="neigh", drop_first=True)
data = pd.concat([data, specific_dummies], axis=1)

**bathrooms_text is in string, words. Should extract something.**

In [469]:
data["bathrooms_text"].unique()

array(['1.5 baths', '1.5 shared baths', '1 private bath', '1 shared bath',
       '1 bath', '2.5 baths', '3.5 baths', '0 baths', 'Private half-bath',
       '2 baths', '3 baths', '0 shared baths', 'Half-bath', nan,
       '5 baths', 'Shared half-bath', '2 shared baths', '5.5 baths',
       '4 baths', '4.5 baths', '4 shared baths', '17 baths',
       '3 shared baths'], dtype=object)

**Converting times to UNIX type, for them to be meaningful**

In [470]:
data['first_review'] = pd.to_datetime(data['first_review'])
data['last_review'] = pd.to_datetime(data['last_review'])
data['host_since'] = pd.to_datetime(data['host_since'])
data['first_review_unix'] = (data['first_review'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
data['last_review_unix'] = (data['last_review'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
data['host_since_unix'] = (data['host_since'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

In [471]:
data['last_review_unix'] = data['last_review_unix'].apply(lambda x: min(data['last_review_unix'][data['last_review_unix']>0]) if x <0 else x)

In [472]:
data['first_review_unix'] = data['first_review_unix'].apply(lambda x: min(data['first_review_unix'][data['first_review_unix']>0]) if x <0 else x)

In [473]:
data['host_since_unix'] = data['host_since_unix'].apply(lambda x: min(data['host_since_unix'][data['host_since_unix']>0]) if x <0 else x)

**Fill NaNs with the minimum of the series**

In [474]:
data["last_review_unix"].fillna(min(data['last_review_unix']), inplace=True)
data["first_review_unix"].fillna(min(data['first_review_unix']), inplace=True)
data["last_review_unix"].fillna(min(data['last_review_unix']), inplace=True)

**Those features are numerical. FIlling their NaNs with zeros. One could fill in with the mean of them, but I deem there is some reason there are NaNs. So I fill in with zeros**

In [475]:
to_zero = ['review_scores_communication', 'host_acceptance_rate', 'review_scores_communication', 'review_scores_checkin', 'review_scores_cleanliness', 'review_scores_location', 'review_scores_value',
           'review_scores_accuracy','review_scores_rating', 'reviews_per_month', 'bedrooms', 'beds', 'host_is_superhost', 'bathrooms_text']
for i in to_zero:
  data[i].fillna(0, inplace=True)


In [476]:
data.drop(columns=["first_review", "last_review"], inplace=True)

**No missing values now. Cool.**

In [477]:
missing_values = data.isna().sum().sort_values(ascending=False)
missing_values = missing_values[missing_values>0]
print(missing_values)

Series([], dtype: int64)


**Change half-bath to numeric value of 0.5**

In [478]:
data['bathrooms_text'] = data['bathrooms_text'].apply(lambda x: str(x).replace("half-bath", "0.5"))
data['bathrooms_text'] = data['bathrooms_text'].apply(lambda x: str(x).replace("Half-bath", "0.5"))

**Get only numbers from the string. So just the number of baths.**

In [479]:
data['bathrooms'] = data['bathrooms_text'].apply(lambda x: re.findall(r'\d+\.\d+|\d+',x)[0])

In [480]:
data['bathrooms']=data['bathrooms'].astype("float")

**Penalize the number of baths, if it is shared**

In [481]:
for index, row in data.iterrows():
    if "share" in row['bathrooms_text']:
      data.at[index, 'bathrooms'] = 0.5*row['bathrooms']

**There are dummies of them, so drop them**

In [482]:
data.drop(columns=["host_since", "bathrooms_text", "property_type", "amenities", "room_type"], inplace=True)

**Aggregate availability. Weight all of them to numeraire **

In [483]:
data["aval"] = (15.66/30)*data["availability_30"] + (15.66/60)*data["availability_60"] + (15.66/90)*data["availability_90"] + (15.66/365)*data["availability_365"]

In [484]:
data.drop(columns=["availability_30", "availability_60", "availability_90", "availability_365"], inplace=True)

**Add data from TripAdvisor for the popular tourist amenities**

In [485]:
# trip = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/PS2/data/december/trip.csv")
trip = pd.read_csv("data/december/trip.csv")

**There are number of reviews from TripAdvisor and Google. Normalize them via max-min**

In [486]:
trip["g"] = (trip["greviews"] - min(trip["greviews"]))/(max(trip["greviews"] - min(trip["greviews"])))

In [487]:
trip["t"] = (trip["treviews"] - min(trip["treviews"]))/(max(trip["treviews"] - min(trip["treviews"])))

**Get the mean score of TripAdvisor and Google**

In [488]:
trip["score"] = 100*((trip["g"]+trip["t"])/2)

**Use the formula $ament_{apartment} = \sum_{amenity} \frac{1}{distance_{apartment, amenity}}\cdot ReviewScore_{amenity}$ to get the index. This index takes into account attractiveness of the place where apartment is, and distance to this from popular amenities** 

In [489]:
for index, row in data.iterrows():
  ament = 0
  for indeks, line in trip.iterrows():
    ament += line['score']/(geopy.distance.distance(np.array(row[['latitude', 'longitude']]), line['location']).km)
  data.at[index, "ament"] = ament

**UNIX time is in seconds. Very large. So I divide to 10^6**

In [490]:
data["first_review_unix"]=data["first_review_unix"]/(10**6)
data["last_review_unix"]=data["last_review_unix"]/(10**6)
data["ament"]=data["ament"]/(10**6)


**Drop unnecessary features. And create lof of the target. Before dropping and did some analysis to check important variables, but I do not show this, omitting for clarity**

In [491]:
drops = ['source', 'latitude', 'longitude', 'neighbourhood_cleansed','minimum_nights', 'maximum_nights',
 'minimum_minimum_nights',
 'maximum_minimum_nights',
 'minimum_maximum_nights',
 'maximum_maximum_nights',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value', 'number_of_reviews_ltm', 'number_of_reviews_l30d',  'calculated_host_listings_count_entire_homes',
 'calculated_host_listings_count_shared_rooms', 'Hot', 'water', 'dryer', 'alarm', 'proptype_Shared room in hostel', 'host_since_unix']
df = data.drop(columns=drops)



In [492]:
df["logprice"]=np.log(df["price"])
df = df.replace([-np.inf], 0)


In [493]:
lista = [ 'accommodates',
 'bedrooms',
 'beds',
 'last_review_unix',
 'bathrooms',
 'aval',
 'ament']
for ind in lista:
  df[ind+str("_2")]=df[ind]*df[ind]

**Create holdout sample**

In [494]:
out_sample = df.sample(n=300)
out_sample_X = out_sample.drop(["logprice", "price", "id", "scrape_id"],axis=1)
out_sample_y =  out_sample["price"]
out_sample_y_log =  out_sample["logprice"]
df_learn = df.drop(out_sample.index)

In [495]:
df_learn_X = df_learn.drop(["logprice", "price", "id", "scrape_id"],axis=1)
y = df_learn["price"]
log_y = df_learn["logprice"]

In [546]:
columns = df_learn_X.columns.to_list()
neigh = filter(lambda x: x.startswith('neigh_'), columns)
room_type = filter(lambda x: x.startswith('room_type_'), columns)
proptype = filter(lambda x: x.startswith('proptype_'), columns)
base = list(['host_acceptance_rate',
 'host_is_superhost',
 'host_listings_count',
 'host_identity_verified',
 'accommodates',
 'bedrooms',
 'beds',
 'minimum_nights_avg_ntm',
 'maximum_nights_avg_ntm',
 'number_of_reviews',
 'review_scores_rating',
 'instant_bookable',
 'calculated_host_listings_count',
 'calculated_host_listings_count_private_rooms',
 'reviews_per_month',
 'Hangers',
 'Dishes',
 'linens',
 'Wifi',
 'Fire',
 'Heating',
 'Refrigerator',
 'premises',
 'Coffee',
 'Smoke',
 'Iron',
 'parking',
 'TV',
 'Kitchen','first_review_unix',
 'last_review_unix',
 'bathrooms',
 'aval',
 'ament'])
quad = list([ 'accommodates_2',
 'bedrooms_2',
 'beds_2',
 'last_review_unix_2',
 'bathrooms_2',
 'aval_2',
 'ament_2'])

In [547]:
from statsmodels.regression.linear_model import OLS

**Four specifications**

In [548]:
formula_1 = list(base)
formula_2 = list(quad)
formula_3 = list(neigh) + list(proptype) + list(room_type)
formula_4 = formula_1 + formula_2 +formula_3

In [549]:
metrics = {}

#the OLS model

def full_ols(dep, formula, name, datas):
    model = OLS(datas[dep],datas[formula])
    results = model.fit()
    metrics[name] = {"RMSE":np.sqrt(results.mse_resid) , "BIC": results.bic}

full_ols('price', formula_1, "model_1", df_learn)
full_ols('price',formula_2, "model_2", df_learn)
full_ols('price',formula_3, "model_3", df_learn)
full_ols('price',formula_4, "model_4", df_learn)

full_ols('logprice', formula_1, "model_1_log", df_learn)
full_ols('logprice',formula_2, "model_2_log", df_learn)
full_ols('logprice',formula_3, "model_3_log", df_learn)
full_ols('logprice',formula_4, "model_4_log", df_learn)

print(metrics)

{'model_1': {'RMSE': 962.1637043608615, 'BIC': 108159.24992701387}, 'model_2': {'RMSE': 963.153457435621, 'BIC': 107962.63425991997}, 'model_3': {'RMSE': 966.2213809951669, 'BIC': 108229.58560671254}, 'model_4': {'RMSE': 962.7532842609536, 'BIC': 108501.43496224325}, 'model_1_log': {'RMSE': 0.4479083734089472, 'BIC': 8280.58202927218}, 'model_2_log': {'RMSE': 0.7102543138695565, 'BIC': 14072.32786809198}, 'model_3_log': {'RMSE': 0.636512499918051, 'BIC': 12870.85836572309}, 'model_4_log': {'RMSE': 0.40647466223555845, 'BIC': 7351.173129348038}}


In [550]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error


In [612]:
k = KFold(n_splits=4, random_state=2023, shuffle=True)

In [625]:
rmse_results = []
for train_index, test_index in k.split(df_learn_X):
    # Get the training and test data for this fold
    X_train, X_test = df_learn_X.iloc[train_index], df_learn_X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    formula = X_train[formula_1]
    y_train = list(y_train)
    model = OLS(y_train,formula)
    results = model.fit()
    # Predict on the test data
    y_pred = results.predict(X_test[formula_1])

    # Calculate the RMSE for this fold
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    rmse_results.append(rmse)
metrics["model_1_k"] = {"RMSE_1":rmse_results[0], "RMSE_2":rmse_results[1], "RMSE_3":rmse_results[2], "RMSE_4":rmse_results[3], "RMSE_average":np.mean(rmse_results)}


In [626]:
rmse_results = []
for train_index, test_index in k.split(df_learn_X):
    # Get the training and test data for this fold
    X_train, X_test = df_learn_X.iloc[train_index], df_learn_X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    formula = X_train[formula_2]
    y_train = list(y_train)
    model = OLS(y_train,formula)
    results = model.fit()
    # Predict on the test data
    y_pred = results.predict(X_test[formula_2])

    # Calculate the RMSE for this fold
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    rmse_results.append(rmse)
metrics["model_2_k"] = {"RMSE_1":rmse_results[0], "RMSE_2":rmse_results[1], "RMSE_3":rmse_results[2], "RMSE_4":rmse_results[3], "RMSE_average":np.mean(rmse_results)}


In [615]:
rmse_results = []
for train_index, test_index in k.split(df_learn_X):
    # Get the training and test data for this fold
    X_train, X_test = df_learn_X.iloc[train_index], df_learn_X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    formula = X_train[formula_3]
    y_train = list(y_train)
    model = OLS(y_train,formula)
    results = model.fit()
    # Predict on the test data
    y_pred = results.predict(X_test[formula_3])

    # Calculate the RMSE for this fold
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    rmse_results.append(rmse)
metrics["model_3_k"] = {"RMSE_1":rmse_results[0], "RMSE_2":rmse_results[1], "RMSE_3":rmse_results[2], "RMSE_4":rmse_results[3], "RMSE_average":np.mean(rmse_results)}


In [616]:
rmse_results = []
for train_index, test_index in k.split(df_learn_X):
    # Get the training and test data for this fold
    X_train, X_test = df_learn_X.iloc[train_index], df_learn_X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    formula = X_train[formula_4]
    y_train = list(y_train)
    model = OLS(y_train,formula)
    results = model.fit()
    # Predict on the test data
    y_pred = results.predict(X_test[formula_4])

    # Calculate the RMSE for this fold
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    rmse_results.append(rmse)
metrics["model_4_k"] = {"RMSE_1":rmse_results[0], "RMSE_2":rmse_results[1], "RMSE_3":rmse_results[2], "RMSE_4":rmse_results[3], "RMSE_average":np.mean(rmse_results)}


In [617]:
rmse_results = []
for train_index, test_index in k.split(df_learn_X):
    # Get the training and test data for this fold
    X_train, X_test = df_learn_X.iloc[train_index], df_learn_X.iloc[test_index]
    y_train, y_test = log_y.iloc[train_index], log_y.iloc[test_index]
    formula = X_train[formula_1]
    y_train = list(y_train)
    model = OLS(y_train,formula)
    results = model.fit()
    # Predict on the test data
    y_pred = results.predict(X_test[formula_1])

    # Calculate the RMSE for this fold
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    rmse_results.append(rmse)
metrics["model_1_k_log"] = {"RMSE_1":rmse_results[0], "RMSE_2":rmse_results[1], "RMSE_3":rmse_results[2], "RMSE_4":rmse_results[3], "RMSE_average":np.mean(rmse_results)}


In [618]:
rmse_results = []
for train_index, test_index in k.split(df_learn_X):
    # Get the training and test data for this fold
    X_train, X_test = df_learn_X.iloc[train_index], df_learn_X.iloc[test_index]
    y_train, y_test = log_y.iloc[train_index], log_y.iloc[test_index]
    formula = X_train[formula_2]
    y_train = list(y_train)
    model = OLS(y_train,formula)
    results = model.fit()
    # Predict on the test data
    y_pred = results.predict(X_test[formula_2])

    # Calculate the RMSE for this fold
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    rmse_results.append(rmse)
metrics["model_2_k_log"] = {"RMSE_1":rmse_results[0], "RMSE_2":rmse_results[1], "RMSE_3":rmse_results[2], "RMSE_4":rmse_results[3], "RMSE_average":np.mean(rmse_results)}


In [619]:
rmse_results = []
for train_index, test_index in k.split(df_learn_X):
    # Get the training and test data for this fold
    X_train, X_test = df_learn_X.iloc[train_index], df_learn_X.iloc[test_index]
    y_train, y_test = log_y.iloc[train_index], log_y.iloc[test_index]
    formula = X_train[formula_3]
    y_train = list(y_train)
    model = OLS(y_train,formula)
    results = model.fit()
    # Predict on the test data
    y_pred = results.predict(X_test[formula_3])

    # Calculate the RMSE for this fold
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    rmse_results.append(rmse)
metrics["model_3_k_log"] = {"RMSE_1":rmse_results[0], "RMSE_2":rmse_results[1], "RMSE_3":rmse_results[2], "RMSE_4":rmse_results[3], "RMSE_average":np.mean(rmse_results)}


In [620]:
rmse_results = []
for train_index, test_index in k.split(df_learn_X):
    # Get the training and test data for this fold
    X_train, X_test = df_learn_X.iloc[train_index], df_learn_X.iloc[test_index]
    y_train, y_test = log_y.iloc[train_index], log_y.iloc[test_index]
    formula = X_train[formula_4]
    y_train = list(y_train)
    model = OLS(y_train,formula)
    results = model.fit()
    # Predict on the test data
    y_pred = results.predict(X_test[formula_4])

    # Calculate the RMSE for this fold
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    rmse_results.append(rmse)
metrics["model_4_k_log"] = {"RMSE_1":rmse_results[0], "RMSE_2":rmse_results[1], "RMSE_3":rmse_results[2], "RMSE_4":rmse_results[3], "RMSE_average":np.mean(rmse_results)}


In [621]:
full_ols('price', formula_1, "model_out_1", out_sample)
full_ols('price', formula_1, "model_out_2", out_sample)
full_ols('price', formula_1, "model_out_3", out_sample)
full_ols('price', formula_4, "model_out_4", out_sample)
full_ols('logprice', formula_1, "model_out_1_log", out_sample)
full_ols('logprice', formula_1, "model_out_2_log", out_sample)
full_ols('logprice', formula_1, "model_out_3_log", out_sample)
full_ols('logprice', formula_4, "model_out_4_log", out_sample)

In [651]:
table = pd.concat([pd.DataFrame(data=[
[metrics["model_1"]["RMSE"], metrics["model_2"]["RMSE"], metrics["model_3"]["RMSE"], metrics["model_4"]["RMSE"]],
[metrics["model_1"]["BIC"], metrics["model_2"]["BIC"], metrics["model_3"]["BIC"], metrics["model_4"]["BIC"]],

[metrics["model_1_log"]["RMSE"], metrics["model_2_log"]["RMSE"], metrics["model_3_log"]["RMSE"], metrics["model_4_log"]["RMSE"]],
[metrics["model_1_log"]["BIC"], metrics["model_2_log"]["BIC"], metrics["model_3_log"]["BIC"], metrics["model_4_log"]["BIC"]],

[metrics["model_1_k"]["RMSE_1"], metrics["model_2_k"]["RMSE_1"], metrics["model_3_k"]["RMSE_1"], metrics["model_4_k"]["RMSE_1"]],
[metrics["model_1_k"]["RMSE_2"], metrics["model_2_k"]["RMSE_2"],metrics["model_3_k"]["RMSE_2"],metrics["model_4_k"]["RMSE_2"]],
[metrics["model_1_k"]["RMSE_3"], metrics["model_2_k"]["RMSE_3"],metrics["model_3_k"]["RMSE_3"],metrics["model_4_k"]["RMSE_3"]],
[metrics["model_1_k"]["RMSE_4"], metrics["model_2_k"]["RMSE_4"],metrics["model_3_k"]["RMSE_4"],metrics["model_4_k"]["RMSE_4"]],
[metrics["model_1_k"]["RMSE_average"], metrics["model_2_k"]["RMSE_average"],metrics["model_3_k"]["RMSE_average"],metrics["model_4_k"]["RMSE_average"]],

[metrics["model_1_k_log"]["RMSE_1"], metrics["model_2_k_log"]["RMSE_1"], metrics["model_3_k_log"]["RMSE_1"], metrics["model_4_k_log"]["RMSE_1"]],
[metrics["model_1_k_log"]["RMSE_2"], metrics["model_2_k_log"]["RMSE_2"],metrics["model_3_k_log"]["RMSE_2"],metrics["model_4_k_log"]["RMSE_2"]],
[metrics["model_1_k_log"]["RMSE_3"], metrics["model_2_k_log"]["RMSE_3"],metrics["model_3_k_log"]["RMSE_3"],metrics["model_4_k_log"]["RMSE_3"]],
[metrics["model_1_k_log"]["RMSE_4"], metrics["model_2_k_log"]["RMSE_4"],metrics["model_3_k_log"]["RMSE_4"],metrics["model_4_k_log"]["RMSE_4"]],
[metrics["model_1_k_log"]["RMSE_average"], metrics["model_2_k_log"]["RMSE_average"],metrics["model_3_k_log"]["RMSE_average"],metrics["model_4_k_log"]["RMSE_average"]],


[metrics["model_out_1"]["RMSE"], metrics["model_out_2"]["RMSE"],metrics["model_out_3"]["RMSE"],metrics["model_out_4"]["RMSE"]],
[metrics["model_out_1"]["BIC"], metrics["model_out_2"]["BIC"],metrics["model_out_3"]["BIC"],metrics["model_out_4"]["BIC"]],

[metrics["model_out_1_log"]["RMSE"], metrics["model_out_2_log"]["RMSE"],metrics["model_out_3_log"]["RMSE"],metrics["model_out_4_log"]["RMSE"]],
[metrics["model_out_1_log"]["BIC"], metrics["model_out_2_log"]["BIC"],metrics["model_out_3_log"]["BIC"],metrics["model_out_4_log"]["BIC"]]

 ], columns=["Model 1", "Model 2", "Model 3", "Model 4"]).rename({0: "RMSE - full sample", 1: "BIC - full sample", 
                                                                  2: "RMSE - full sample log", 3: "BIC - full sample log", 
                                                                  4:"Fold 1", 5:"Fold 2", 6:"Fold 3", 7:"Fold 4", 8:"Average RMSE", 
                                                                  
                                                                  9:"Fold 1 log", 10:"Fold 2 log", 11:"Fold 3 log", 12:"Fold 4 log", 13:"Average RMSE log", 
                                                                  14:"Out sample RMSE", 15:"Out sample BIC",
                                                                   16:"Out sample RMSE log", 17:"Out sample BIC log"
                                                                   }, axis=0),
    ]).round(3)
print(table)

                            Model 1     Model 2     Model 3      Model 4
RMSE - full sample          962.164     963.153     966.221      962.753
BIC - full sample        108159.250  107962.634  108229.586   108501.435
RMSE - full sample log        0.448       0.710       0.637        0.406
BIC - full sample log      8280.582   14072.328   12870.858     7351.173
Fold 1                      264.066     259.283     271.969      273.063
Fold 2                  1126194.321     156.485     163.930  1142422.561
Fold 3                     1774.156    1775.863    1775.575     1783.805
Fold 4                      702.791     695.494     697.673      705.836
Average RMSE             282233.833     721.781     727.287   286296.316
Fold 1 log                    0.481       0.734       0.658        0.441
Fold 2 log                 1325.175       1.012       0.685     1137.202
Fold 3 log                    0.449       0.874       0.605        0.638
Fold 4 log                    0.443       0.698    

**By RMSE on full sample Model 1 is the best, but considering BIC Model 2 is the best one. On log scale Model 4 is the best one, with BIC also. CV shows that Model 2 and Model 3 are the best ones, in the log scale target also.Holdout results differ.**

In [393]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso 
from sklearn import metrics 

In [420]:
lasso = Lasso()
params = {'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 
 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0, 
 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50, 100, 500, 1000, 2000, 3000]}
folds = 4
# cross validation
model_cv = GridSearchCV(estimator = lasso, 
                        param_grid = params, 
                        scoring= 'neg_mean_absolute_error', 
                        cv = 4)            

model_cv.fit(df_learn_X[formula_1], y) 

cv_results = pd.DataFrame(model_cv.cv_results_)

cv_results.sort_values(by='mean_test_score', ascending=False)["param_alpha"][:2]

29    3000
2     0.01
Name: param_alpha, dtype: object

In [421]:
lasso = Lasso()
params = {'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 
 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0, 
 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50, 100, 500, 1000, 2000, 3000]}
folds = 4
# cross validation
model_cv = GridSearchCV(estimator = lasso, 
                        param_grid = params, 
                        scoring= 'neg_mean_absolute_error', 
                        cv = 4)            

model_cv.fit(df_learn_X[formula_2], y) 

cv_results = pd.DataFrame(model_cv.cv_results_)

cv_results.sort_values(by='mean_test_score', ascending=False)["param_alpha"][:2]

0    0.0001
1     0.001
Name: param_alpha, dtype: object

In [422]:
lasso = Lasso()
params = {'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 
 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0, 
 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50, 100, 500, 1000, 2000, 3000]}
folds = 4
# cross validation
model_cv = GridSearchCV(estimator = lasso, 
                        param_grid = params, 
                        scoring= 'neg_mean_absolute_error', 
                        cv = 4)            

model_cv.fit(df_learn_X[formula_3], y) 

cv_results = pd.DataFrame(model_cv.cv_results_)

cv_results.sort_values(by='mean_test_score', ascending=False)["param_alpha"][:2]

13    1.0
11    0.8
Name: param_alpha, dtype: object

In [423]:
lasso = Lasso()
params = {'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 
 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0, 
 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50, 100, 500, 1000, 2000, 3000]}
folds = 4
# cross validation
model_cv = GridSearchCV(estimator = lasso, 
                        param_grid = params, 
                        scoring= 'neg_mean_absolute_error', 
                        cv = 4)            

model_cv.fit(df_learn_X[formula_4], y) 

cv_results = pd.DataFrame(model_cv.cv_results_)

cv_results.sort_values(by='mean_test_score', ascending=False)["param_alpha"][:2]

29    3000
28    2000
Name: param_alpha, dtype: object

In [426]:
lasso = Lasso()
params = {'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 
 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0, 
 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50, 100, 500, 1000, 2000, 3000]}
folds = 4
# cross validation
model_cv = GridSearchCV(estimator = lasso, 
                        param_grid = params, 
                        scoring= 'neg_mean_absolute_error', 
                        cv = 4)            

model_cv.fit(df_learn_X[formula_1], log_y) 

cv_results = pd.DataFrame(model_cv.cv_results_)

cv_results.sort_values(by='mean_test_score', ascending=False)["param_alpha"][:2]

29    3000
28    2000
Name: param_alpha, dtype: object

In [427]:
lasso = Lasso()
params = {'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 
 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0, 
 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50, 100, 500, 1000, 2000, 3000]}
folds = 4
# cross validation
model_cv = GridSearchCV(estimator = lasso, 
                        param_grid = params, 
                        scoring= 'neg_mean_absolute_error', 
                        cv = 4)            

model_cv.fit(df_learn_X[formula_2], log_y) 

cv_results = pd.DataFrame(model_cv.cv_results_)

cv_results.sort_values(by='mean_test_score', ascending=False)["param_alpha"][:2]

0    0.0001
1     0.001
Name: param_alpha, dtype: object

In [428]:
lasso = Lasso()
params = {'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 
 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0, 
 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50, 100, 500, 1000, 2000, 3000]}
folds = 4
# cross validation
model_cv = GridSearchCV(estimator = lasso, 
                        param_grid = params, 
                        scoring= 'neg_mean_absolute_error', 
                        cv = 4)            

model_cv.fit(df_learn_X[formula_3], log_y) 

cv_results = pd.DataFrame(model_cv.cv_results_)

cv_results.sort_values(by='mean_test_score', ascending=False)["param_alpha"][:2]

1     0.001
0    0.0001
Name: param_alpha, dtype: object

In [429]:
lasso = Lasso()
params = {'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 
 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0, 
 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50, 100, 500, 1000, 2000, 3000]}
folds = 4
# cross validation
model_cv = GridSearchCV(estimator = lasso, 
                        param_grid = params, 
                        scoring= 'neg_mean_absolute_error', 
                        cv = 4)            

model_cv.fit(df_learn_X[formula_4], log_y) 

cv_results = pd.DataFrame(model_cv.cv_results_)

cv_results.sort_values(by='mean_test_score', ascending=False)["param_alpha"][:2]

29    3000
28    2000
Name: param_alpha, dtype: object

In [434]:
formulas = [formula_1, formula_2, formula_3, formula_4]
y_alpha = [0.01, 0.001, 0.8, 2000]
ylog_alpha = [2000, 0.001,0.001,2000]
n=0
for i in y_alpha:
  lasso = Lasso(alpha=i)
  lasso.fit(df_learn_X[formulas[n]],  y)
  y_pred = lasso.predict(out_sample_X[formulas[n]])
  n += 1
  print(np.sqrt(metrics.mean_squared_error(out_sample_y,y_pred)))

148.0293757579665
150.4923232905115
154.14600700112607
160.70124533576086


In [435]:
n=0
for i in ylog_alpha:
  lasso = Lasso(alpha=i)
  lasso.fit(df_learn_X[formulas[n]],  log_y)
  y_pred = lasso.predict(out_sample_X[formulas[n]])
  n += 1
  print(np.sqrt(metrics.mean_squared_error(out_sample_y_log,y_pred)))

0.5576329220871954
0.5519331212161148
0.4629085168955835
0.5567455219323949


In [437]:
from xgboost import XGBRegressor


In [639]:
xgb1 = XGBRegressor(seed = 2023)
parameters = {'nthread':[-1],
              'objective':['reg:squarederror'],
              'learning_rate': [.03, .07],
              'max_depth': [5, 10],
              'silent': [1],
              'subsample': [0.85],
              'colsample_bytree': [0.5],
              'n_estimators': [500]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 4,
                        n_jobs = -1,
                        scoring="neg_root_mean_squared_error")

xgb_grid.fit(df_learn_X[formula_1],y)

# print(xgb_grid.best_score_)
# print(xgb_grid.best_params_)

print(np.sqrt(mean_squared_error(out_sample_y, xgb_grid.predict(out_sample_X[formula_1]))))

127.51394744427928


In [640]:
xgb1 = XGBRegressor(seed = 2023)
parameters = {'nthread':[-1],
              'objective':['reg:squarederror'],
              'learning_rate': [.03, .07],
              'max_depth': [5, 10],
              'silent': [1],
              'subsample': [0.85],
              'colsample_bytree': [0.5],
              'n_estimators': [500]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 4,
                        n_jobs = -1,
                        scoring="neg_root_mean_squared_error")

xgb_grid.fit(df_learn_X[formula_2],y)
print(np.sqrt(mean_squared_error(out_sample_y, xgb_grid.predict(out_sample_X[formula_2]))))

139.65707500696715


In [641]:
xgb1 = XGBRegressor(seed = 2023)
parameters = {'nthread':[-1],
              'objective':['reg:squarederror'],
              'learning_rate': [.03, .07],
              'max_depth': [5, 10],
              'silent': [1],
              'subsample': [0.85],
              'colsample_bytree': [0.5],
              'n_estimators': [500]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 4,
                        n_jobs = -1,
                        scoring="neg_root_mean_squared_error")

xgb_grid.fit(df_learn_X[formula_3],y)
print(np.sqrt(mean_squared_error(out_sample_y, xgb_grid.predict(out_sample_X[formula_3]))))

180.64360955690316


In [642]:
xgb1 = XGBRegressor(seed = 2023)
parameters = {'nthread':[-1],
              'objective':['reg:squarederror'],
              'learning_rate': [.03, .07],
              'max_depth': [5, 10],
              'silent': [1],
              'subsample': [0.85],
              'colsample_bytree': [0.5],
              'n_estimators': [500]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 4,
                        n_jobs = -1,
                        scoring="neg_root_mean_squared_error")

xgb_grid.fit(df_learn_X[formula_4],y)
print(np.sqrt(mean_squared_error(out_sample_y, xgb_grid.predict(out_sample_X[formula_4]))))

118.02371153809318


In [643]:
xgb1 = XGBRegressor(seed = 2023)
parameters = {'nthread':[-1],
              'objective':['reg:squarederror'],
              'learning_rate': [.03, .07],
              'max_depth': [5, 10],
              'silent': [1],
              'subsample': [0.85],
              'colsample_bytree': [0.5],
              'n_estimators': [500]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 4,
                        n_jobs = -1,
                        scoring="neg_root_mean_squared_error")

xgb_grid.fit(df_learn_X[formula_1],log_y)
print(np.sqrt(mean_squared_error(out_sample_y_log, xgb_grid.predict(out_sample_X[formula_1]))))

0.32226302998271317


In [644]:
xgb1 = XGBRegressor(seed = 2023)
parameters = {'nthread':[-1],
              'objective':['reg:squarederror'],
              'learning_rate': [.03, .07],
              'max_depth': [5, 10],
              'silent': [1],
              'subsample': [0.85],
              'colsample_bytree': [0.5],
              'n_estimators': [500]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 4,
                        n_jobs = -1,
                        scoring="neg_root_mean_squared_error")

xgb_grid.fit(df_learn_X[formula_2],log_y)
print(np.sqrt(mean_squared_error(out_sample_y_log, xgb_grid.predict(out_sample_X[formula_2]))))

0.3730223366657394


In [645]:
xgb1 = XGBRegressor(seed = 2023)
parameters = {'nthread':[-1],
              'objective':['reg:squarederror'],
              'learning_rate': [.03, .07],
              'max_depth': [5, 10],
              'silent': [1],
              'subsample': [0.85],
              'colsample_bytree': [0.5],
              'n_estimators': [500]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 4,
                        n_jobs = -1,
                        scoring="neg_root_mean_squared_error")

xgb_grid.fit(df_learn_X[formula_3],log_y)
print(np.sqrt(mean_squared_error(out_sample_y_log, xgb_grid.predict(out_sample_X[formula_3]))))

0.44644560390678417


In [646]:
xgb1 = XGBRegressor(seed = 2023)
parameters = {'nthread':[-1],
              'objective':['reg:squarederror'],
              'learning_rate': [.03, .07],
              'max_depth': [5, 10],
              'silent': [1],
              'subsample': [0.85],
              'colsample_bytree': [0.5],
              'n_estimators': [500]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 4,
                        n_jobs = -1,
                        scoring="neg_root_mean_squared_error")

xgb_grid.fit(df_learn_X[formula_4],log_y)
print(np.sqrt(mean_squared_error(out_sample_y_log, xgb_grid.predict(out_sample_X[formula_4]))))

0.3172774335626686


In [652]:
table = pd.concat([pd.DataFrame(data=[
[metrics["model_out_1"]["RMSE"], metrics["model_out_2"]["RMSE"],metrics["model_out_3"]["RMSE"],metrics["model_out_4"]["RMSE"]],

[metrics["model_out_1_log"]["RMSE"], metrics["model_out_2_log"]["RMSE"],metrics["model_out_3_log"]["RMSE"],metrics["model_out_4_log"]["RMSE"]],
["148", "150.5", "154.2", "160.7" ],
["0.558", "0.551", "046", "0.557" ],
["127.51", "139.65", "180.64", "118" ],
["0.322", "0.373", "0.44", "0.317" ]

 ], columns=["Model 1", "Model 2", "Model 3", "Model 4"]).rename({ 0:"RMSE: Out sample OLS price", 
                                                                   1:"RMSE: Out sample OLS log price", 
                                                                   2: "RMSE: Out Sample: Lasso Price",
                                                                   3: "RMSE: Out Sample: Lasso Log Price", 
                                                                   4: "RMSE: Out Sample: XGB Price", 
                                                                   5: "RMSE: Out Sample: XGB Log Price", 
                                                                   }, axis=0),
    ]).round(3)
print(table)

                                      Model 1     Model 2     Model 3  \
RMSE: Out sample OLS price         134.201855  134.201855  134.201855   
RMSE: Out sample OLS log price       0.413124    0.413124    0.413124   
RMSE: Out Sample: Lasso Price             148       150.5       154.2   
RMSE: Out Sample: Lasso Log Price       0.558       0.551         046   
RMSE: Out Sample: XGB Price            127.51      139.65      180.64   
RMSE: Out Sample: XGB Log Price         0.322       0.373        0.44   

                                     Model 4  
RMSE: Out sample OLS price         91.661515  
RMSE: Out sample OLS log price      0.343681  
RMSE: Out Sample: Lasso Price          160.7  
RMSE: Out Sample: Lasso Log Price      0.557  
RMSE: Out Sample: XGB Price              118  
RMSE: Out Sample: XGB Log Price        0.317  


**In OLS Model 4 wins. In Lasso Model 1 for price, and Model 4 for log price. For XGB also Model 4 wins. Seemingly, althoug there is evidence of overfitting in OLS Model 4, because of many features it did best in hold out data as other types of models**